#Anchors
Anchortabular takes into consideration the whole dataset. It understands it globally and then gives a local explanation. Therefore the accuracy of predicting a feature's importance

It is very similar to LIME but the latter adds noise from the normal distribution and does not consider the global datapoints

In [1]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt
!pip install alibi

     |████████████████████████████████| 225kB 2.9MB/s 
     |████████████████████████████████| 327kB 8.9MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 204kB 13.2MB/s 
     |████████████████████████████████| 28.0MB 169kB/s 
  Created wheel for shap: filename=shap-0.36.0-cp36-cp36m-linux_x86_64.whl size=456461 sha256=56b82d257c9ed8c93665c1cf7df44f43535bd95f675e1b6cde3376076528c737
  Stored in directory: /root/.cache/pip/wheels/fb/15/e1/8f61106790da27e0765aaa6e664550ca2c50ea339099e799f4
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.1.0-py2.py3-none-any.whl size=28052144 sha256=e9d1c8b443dd9961fc4a615d1d3a48b5bd269ad634fbb9d111595d5a1975c0c4
  Stored in directory: /root/.cache/pip/wheels/2a/2b/0a/d6fb6235c56d014d224bca760d15d7cbdd820813085ffcd35d
Successfully built shap spacy-lookups-data


Loading the Breast cancer dataset 

In [2]:
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
data = dataset.data
target = dataset.target
feature_names = dataset.feature_names

In [3]:
from sklearn import preprocessing
X = np.asarray(data)
y = np.asarray(target)
print(len(X),len(y))

569 569


Splitting the data into training and testing set

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (455, 30) (455,)
Test set: (114, 30) (114,)


Using Logistic Regression model

In [5]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(C=0.01,solver='liblinear',max_iter=500)
model

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
model.fit(X_train,y_train)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Importing AnchorTabular for local interpretation:

In [7]:
from alibi.explainers import AnchorTabular
predict_func=lambda x:model.predict_proba(x)
explainer=AnchorTabular(predict_func,feature_names)

In [8]:
model.predict_proba(X_test)

array([[6.66120814e-01, 3.33879186e-01],
       [3.33569206e-03, 9.96664308e-01],
       [1.00000000e+00, 1.25284472e-13],
       [9.99978658e-01, 2.13419080e-05],
       [8.27991629e-01, 1.72008371e-01],
       [9.39564972e-01, 6.04350279e-02],
       [7.24543677e-03, 9.92754563e-01],
       [1.96793049e-03, 9.98032070e-01],
       [5.84706145e-01, 4.15293855e-01],
       [8.45442958e-03, 9.91545570e-01],
       [1.00000000e+00, 2.99392207e-11],
       [5.11656809e-02, 9.48834319e-01],
       [3.25585662e-01, 6.74414338e-01],
       [1.28521526e-01, 8.71478474e-01],
       [9.22384212e-03, 9.90776158e-01],
       [2.44362422e-03, 9.97556376e-01],
       [5.07459077e-03, 9.94925409e-01],
       [2.56338686e-01, 7.43661314e-01],
       [2.83377988e-03, 9.97166220e-01],
       [5.89590250e-03, 9.94104098e-01],
       [1.30603709e-02, 9.86939629e-01],
       [8.05147960e-02, 9.19485204e-01],
       [9.99994069e-01, 5.93106262e-06],
       [1.30143575e-02, 9.86985643e-01],
       [5.862479

In [9]:
explainer.fit(X_train)

AnchorTabular(meta={
    'name': 'AnchorTabular',
    'type': ['blackbox'],
    'explanations': ['local'],
    'params': {'seed': None, 'disc_perc': (25, 50, 75)}
})

In [10]:
from sklearn.metrics import jaccard_similarity_score
yjc = model.predict(X_test)
jaccard_similarity_score(y_test, yjc)

jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.


0.8508771929824561

In [11]:
inst=1
target_label=['Benign','Malignant']
print('Cancer is',target_label[explainer.predictor(X_test[inst].reshape(1, -1))[0]])
anchor=explainer.explain(X_test[inst])
print('Anchor generated feature(/s)',anchor.anchor)

Cancer is Malignant
Anchor generated feature(/s) ['worst area <= 516.45']


Now seeing the feature(/s) due to which the prediction takes place

In [ ]:
anchor=explainer.explain(X_test[inst])
print('Anchor generated feature(/s)',anchor.anchor)

Anchor generated feature(/s) ['worst area <= 516.45']


We can see that the if feature 'worst area' is less than or equal to 516.45, then it has lead to the malignant prediction

Taking another instance where condition is benign, and then finding out the anchors

In [ ]:
print('Cancer is',target_label[explainer.predictor(X_test[5].reshape(1, -1))[0]])
anchor=explainer.explain(X_test[8])
print('Anchor generated feature(/s)',anchor.anchor)

Cancer is Benign
Anchor generated feature(/s) ['worst area > 688.60', 'mean radius <= 15.81', 'texture error > 1.48']
